In [0]:


from sodapy import Socrata
import pandas as pd

# Token y dataset desde widgets y secrets
token = dbutils.secrets.get("claves", "token_app")
dbutils.widgets.text("codigo_dataset", "")
codigo_dataset = dbutils.widgets.get("codigo_dataset")

# Cliente Socrata autenticado
client = Socrata("www.datos.gov.co", str(token))

In [0]:
from pyspark.sql import SparkSession


In [0]:
# ---------------------------------------------
limit = 100000       # Número de registros por página (ajustable)
offset = 0           # Comienza en 0
write_mode = "overwrite"  # Solo overwrite en la primera escritura
total_registros = 0  # Contador acumulado

# ✅ Crear la sesión Spark UNA sola vez al inicio
from pyspark.sql import SparkSession
df_spark = spark.createDataFrame(df_pd)



# ---------------------------------------------
# Nombre de la tabla de destino
# ---------------------------------------------
nombre_tabla = "main.diplomado_datos.ids_contratos_procesos"

print(f'🚀 Iniciando la extracción de datos desde el dataset: {codigo_dataset}')

# ---------------------------------------------
# Bucle principal de paginación
# ---------------------------------------------
while True:
    print(f'\n📦 Descargando registros desde offset: {offset}')

    # Construir la consulta SoQL
    query = f"""
    SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad, documento_proveedor
    LIMIT {limit}
    OFFSET {offset}
    """

    try:
        results = client.get(codigo_dataset, query=query)
    except Exception as e:
        print(f"❌ Error en la descarga desde offset {offset}: {e}")
        break

    if not results:
        print("✅ No hay más registros para descargar.")
        break

    try:
        # Convertir resultados a DataFrame de pandas
        df_pd = pd.DataFrame.from_records(results)

        if df_pd.empty:
            print("📭 Bloque vacío, terminando.")
            break

        # Validar y reactivar SparkSession si fue cerrada por timeout
        spark = SparkSession.builder.getOrCreate()

        # Crear DataFrame de Spark desde pandas
        df_spark = spark.createDataFrame(df_pd)

        # Escribir el bloque a tabla Delta
        df_spark.write \
            .format("delta") \
            .mode(write_mode) \
            .saveAsTable(nombre_tabla)

        print(f"✔ Guardados {len(df_pd)} registros.")
        total_registros += len(df_pd)
        print(f"📊 Total acumulado: {total_registros}")

        # Avanzar al siguiente bloque
        offset += limit
        write_mode = "append"  # Cambiar el modo luego del primer write

        # Evitar throttling
        time.sleep(1)

    except Exception as e:
        print(f"❌ Error procesando bloque en offset {offset}: {e}")
        break

print(f"\n🏁 Proceso finalizado. Total registros descargados: {total_registros}")

In [0]:


# Parámetros de descarga
limit = 100000  # recomendable por estabilidad
offset = 0
write_mode = "overwrite"
total_registros_descargados = 0

print(f'Iniciando la extracción de datos del dataset {codigo_dataset}')

# Loop con paginación
while True:
    print(f'➤ Descargando desde offset: {offset}')
    
    query = f"""
    SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad, documento_proveedor
    LIMIT {limit}
    OFFSET {offset}
    """
    
    # Obtener datos
    results = client.get(codigo_dataset, query=query)
    
    if not results:
        print("✔ No hay más registros para descargar.")
        break
    
    # Convertir a Pandas y luego a Spark
    df_pd = pd.DataFrame.from_records(results)
    df_spark = spark.createDataFrame(df_pd)
    
    # Escribir a Delta
    df_spark.write \
        .format("delta") \
        .mode(write_mode) \
        .saveAsTable("main.diplomado_datos.ids_contratos_procesos")

    # Cambiar modo de escritura a append
    write_mode = "append"
    offset += limit
    total_registros_descargados += len(results)
    print(f'✔ Total acumulado: {total_registros_descargados}')

print("✅ Descarga completa.")


In [0]:
# Databricks notebook source
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata
 
#client = Socrata("www.datos.gov.co", None)
 
token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                  str(token))
 
query="""
Select numero_del_contrato,numero_de_proceso, nit_de_la_entidad, documento_proveedor
limit 20000000
"""
 
 
results = client.get(codigo_dataset, query=query)
 
 
# COMMAND ----------
 
results_df = spark.createDataFrame(results)
 
print(results_df.schema)
 
# COMMAND ----------
 
results_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.diplomado_datos.ids_contratos_procesos")